In [8]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient



from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################


username = "aacuser"
password = "animals"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Natalie Dykhuis SNHU CS-340'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    dcc.Dropdown(
        id='filter_animal',
        options=[
            {'label': 'Cat', 'value': '"animal_type":"Cat"'},
            {'label': 'Dog', 'value': '"animal_type":"Dog"'},
            {'label': 'Other', 'value': '"animal_type":"Other"'}],
        style={'width':'75%'}
    ),
    dcc.RadioItems(
        id='radio',
        options=[
            {'label': 'Water', 'value': 'water'},
            {'label': 'Mountain or Wilderness', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value':'reset'}],
        style={'width':'75%'}
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #data=df.to_dict('records'),
        page_action="native",
        page_current= 10,
        page_size= 10,
        
       
    ),
    dcc.Graph(id="bar-chart"),
    html.Br(),
    html.Hr(),
    html.Div(
           id='map-id',
           className='col s12 m6',
            ),
    dl.Map(dl.TileLayer(),  
    style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block", "position": "relative"})
   
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('table-multicol-sorting', "data"),
    Input('table-multicol-sorting', "page_current"),
    Input('table-multicol-sorting', "page_size"),
    Input('table-multicol-sorting', "sort_by"))
def update_table(page_current, page_size, sort_by):
    print(sort_by)
    if len(sort_by):
        dff = df.sort_values(
            [col['column_id'] for col in sort_by],
            ascending=[
                col['direction'] == 'asc'
                for col in sort_by
            ],
            inplace=False
        )
    else:
        # No sort is applied
        dff = df

    return dff.iloc[
        page_current*page_size:(page_current+ 1)*page_size
    ].to_dict('records')
	
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
@app.callback(
    Output("bar-chart", "figure"), 
    [Input('filter_animal','value'),Input('radio','value')])
def update_bar_chart(v2,v1):
    if(v1 == 'water'):
        df=pd.DataFrame(list(shelter.read({v2,"animal_type":"Dog",
        "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
        "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},
        "age_upon_outcome_in_weeks":{"$lte":156}})))
        
       
    if(v1 == 'mountain'):
        df=pd.DataFrame(list(shelter.read({v2,"animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Alaskan Malamute"," Old English Sheepdog","Siberian Husky","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},
        "age_upon_outcome_in_weeks":{"$lte":156}})))  
        
    if(v1 == 'disaster'):
        df=pd.DataFrame(list(shelter.read({v2,"animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Doberman Pinscher","Golden Retriever","Bloodhound","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},
        "age_upon_outcome_in_weeks":{"$lte":300}})))
        return df.to_dict('records')
    if(value == 'reset'):
        df=pd.DataFrame(list(shelter.read({})))
        return df.to_dict('records')
    
    fig = px.bar(df[mask], x='breed', y = entries in data)
    return fig
   
    
@app.callback(Output('datatable-interactivity',"data"),
             Input('filter_animal','value'))

def Filter_animals(value):
    df=pd.DataFrame(list(shelter.read(value)))
    return df.to_dict('records')

@app.callback(Output('datatable-interactivity',"data"),
             Input('radio','value'))
def Filter_Rescue(value):
    if(value == 'water'):
        df=pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
        "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},
        "age_upon_outcome_in_weeks":{"$lte":156}})))
        return df.to_dict('records')
    if(value == 'mountain'):
        df=pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Alaskan Malamute"," Old English Sheepdog","Siberian Husky","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},
        "age_upon_outcome_in_weeks":{"$lte":156}})))  
        return df.to_dict('records')
    if(value == 'disaster'):
        df=pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Doberman Pinscher","Golden Retriever","Bloodhound","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},
        "age_upon_outcome_in_weeks":{"$lte":300}})))
        return df.to_dict('records')
    if(value == 'reset'):
        df=pd.DataFrame(list(shelter.read({})))
        return df.to_dict('records')

app


SyntaxError: invalid syntax (<ipython-input-8-5da4d9e87f2d>, line 154)